In [1]:
import numpy as np
import cv2
import pickle
import sklearn

In [2]:
# Face detection
face_detect_model = cv2.dnn.readNetFromCaffe("../models/deploy.prototxt.txt", 
                                             "../models/res10_300x300_ssd_iter_140000.caffemodel")

face_cascade = cv2.CascadeClassifier("../cascade_classifier/haarcascade_frontalface_default.xml")

# Feature extraction
face_feature_model = cv2.dnn.readNetFromTorch("../models/openface.nn4.small2.v1.t7")

# Face recognition
face_recognition_model = pickle.load(open('../models/machinelearning_face_person_identity.pkl', mode='rb'))

# Emotion recognition
emotion_recognition_model = pickle.load(open('../models/machinelearning_face_emotion.pkl', mode='rb'))

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. Fo

In [3]:
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
def pipeline(img_path):
    img = cv2.imread(img_path)
    image = img.copy()
    h, w = img.shape[:2]
    img_blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB=False, crop=False)
    face_detect_model.setInput(img_blob)
    detections = face_detect_model.forward()
    
    machinelearning_results = dict(face_detect_score = [],
                                  face_name = [],
                                  face_name_score = [],
                                  emotion_name = [],
                                  emotion_name_score = [],
                                  count = [])

    count = 1 
    if len(detections) > 0:
        for i, confidence in enumerate(detections[0, 0, :, 2]):
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7]*np.array([w, h, w, h])
                sx, sy, ex, ey = box.astype(int)
                cv2.rectangle(image, (sx, sy), (ex, ey), (255, 0, 0))

                # feature extraction 
                face_roi = img[sy:ey, sx:ex]
                face_blob = cv2.dnn.blobFromImage(face_roi, 1/255, (96, 96), (0, 0, 0), swapRB=True, crop=True)
                face_feature_model.setInput(face_blob)
                vectors = face_feature_model.forward()

                #predict with ml model
                face_name = face_recognition_model.predict(vectors)[0]
                face_score = face_recognition_model.predict_proba(vectors).max()

                #emotion
                face_emotion = emotion_recognition_model.predict(vectors)[0]
                face_emotion_score = emotion_recognition_model.predict_proba(vectors).max()

                face_text = "{}, {:.0f}%".format(face_name, 100*face_score)
                emotion_text = "{}, {:.2f}%".format(face_emotion, 100*face_emotion_score)
                cv2.putText(image, face_text, (sx, sy), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
                cv2.putText(image, emotion_text, (sx, ey), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
                
                machinelearning_results['count'].append(count)
                machinelearning_results['face_detect_score'].append(confidence)
                machinelearning_results['face_name'].append(face_name)
                machinelearning_results['face_name_score'].append(face_score)
                machinelearning_results['emotion_name'].append(face_emotion)
                machinelearning_results['emotion_name_score'].append(face_emotion_score)
                count += 1
                
    return image
#     return machinelearning_results

In [5]:
new = pipeline("../data/joe_trump_obama.jpg")
show("WOW", new)